In [56]:
from getpass import getpass
import datetime as dt
from datetime import datetime

import pandas as pandas
import numpy as np
import json

# Mongo
from pymongo import MongoClient
from bson import ObjectId

# Custom backend
import psycopg2 as pg
import PostgresDriver as pd
import binary

from ewdetect.data import *
from ewdetect.preprocessing import *
from ewdetect.intermediates import *

from tqdm import tqdm

In [2]:
stage_connection = connect_pg( 'sshah99', backend = 'stage' )
prod_connection = connect_pg( 'sshah99', backend = 'prod' )

stage_driver, stage_s3_driver = connect_s3( backend = 'stage' )
prod_driver, prod_s3_driver = connect_s3( backend = 'prod' )

Password for pgSQL instance at epiwatch-stage.cxoyodq5cuh1.us-east-1.rds.amazonaws.com
········
Success
Password for pgSQL instance at epiwatch.cxoyodq5cuh1.us-east-1.rds.amazonaws.com
········
Success
Password for PostgresDriver connection (stage)
········
Success
Password for PostgresDriver connection (prod)
········
Success


In [80]:
seiz_log = pandas.read_excel('GTCS_test.xlsx')

seiz_log[['False Negative', 'Bad Seizure', 'Atypical']] = seiz_log[['False Negative', 'Bad Seizure', 'Atypical']].fillna(value='N')

AttributeError: module 'PostgresDriver' has no attribute 'read_excel'

In [14]:
# cur_run = make_new_run()

cur_run = get_latest_run()

In [16]:
json_data = []
for i in range(len(seiz_log)): 
    data = seiz_log.iloc[i][4]
    false_neg = seiz_log.iloc[i][7]
    bad_sz = seiz_log.iloc[i][8]
    atyp = seiz_log.iloc[i][9]
    backend = seiz_log.iloc[i][10]
    
    sz_descriptor = {
        'dataSource': backend,
        'faked': True if bad_sz == 'Y' else False,
        'falseNegative': True if false_neg == 'Y' else False,
        'atypical': True if atyp == 'Y' else False
    }
    
    db_kind = backend.split( '-' )[0]
    if db_kind == 'mongo':
        split_dat = data.split('&')
        start = split_dat[4][3:]
        end = split_dat[5][3:]

        uid = split_dat[0].split('?')[1][3:]
        
        sz_descriptor['userId'] =  uid
        sz_descriptor['start'] =  start
        sz_descriptor['end'] =  end
        
        data_acc = (np.array([]), np.array([]))
        data_hr = (np.array([]), np.array([]))
    
    if db_kind == 'pg':
        db_location = backend.split( '-' )[1]
        
        split_dat = data.split(' ')
        
        uid = split_dat[2]
        start = split_dat[7][2:] + ' ' + split_dat[8][:-2]
        end = split_dat[9][1:] + ' ' + split_dat[10][:-2]
        
        sz_descriptor['userId'] =  uid
        sz_descriptor['start'] =  start
        sz_descriptor['end'] =  end

        
        link_query = ("select * from time_series_index where user_id = {0} " +
                      "and (started, ended) overlaps ('{1}', '{2}') " +
                      "order by started desc").format( uid, start, end)
        
        if db_location == 'prod':
            s3_links = prod_driver.queryRequest( link_query )
            data_collection = prod_s3_driver.MakeS3Requests( s3_links )
            
        elif db_location == 'stage':
            s3_links = stage_driver.queryRequest( link_query )
            data_collection = stage_s3_driver.MakeS3Requests( s3_links )
        
        data_acc, data_hr = (reformat_timeseries_pg( data_collection.epiSensorData.accelerometerData ),
                reformat_hr_pg( data_collection.epiSensorData.heartRateData ))
        
    t_acc, v_acc = data_acc[0], data_acc[1]
    t_hr, v_hr = data_hr[0], data_hr[1]
    
    export_data_seizure = {
        'descriptor': sz_descriptor,
        'data': {
            'accelerometer': {
                't': t_acc,
                'v': v_acc
            },
            'heartRate': {
                't': t_hr,
                'v': v_hr
            }
        }
    }
    
    save_intermediates( export_data_seizure, 'seizure-data-raw',
                        run = cur_run,
                        user = sz_descriptor['userId'],
                        segment = i,
                        nested = True )
    
    json_data.append(sz_descriptor)
    
with open('seizures_raw.json', 'wt') as sz_json:
    json.dump(json_data, sz_json, indent=4, separators=(',', ': '))

number of payloads: 5
Done.
number of payloads: 11
Done.
number of payloads: 11
Done.
number of payloads: 16
Done.
number of payloads: 12
Done.
number of payloads: 10
Done.
number of payloads: 6
Done.
number of payloads: 5
Done.
number of payloads: 8
Done.
number of payloads: 7
Done.
number of payloads: 17
Done.
number of payloads: 5
Done.
number of payloads: 10
Done.
number of payloads: 11
Done.
number of payloads: 11
Done.
number of payloads: 12
Done.


In [54]:
sz_timestamps = []

SQL_TIMESTAMP = "%Y-%m-%d %H:%M:%S"

cur = stage_driver.conn.cursor()

seizures_filename = 'seizures_raw.json'

seizure_descriptors = None
with open( seizures_filename, 'r' ) as seizures_file:
    seizure_descriptors = json.load( seizures_file )
    
for ind, i in enumerate(seizure_descriptors):
    if ind > 5:

        start_time = i["start"]
        end_time = i["end"]

        Query = "SELECT user_id, time_of_detection FROM seizure_detection WHERE time_of_detection BETWEEN '{}' AND '{}' ORDER BY time_of_detection ASC".format(start_time, end_time)
        cur.execute(Query)

        all_schemas = cur.fetchall()

#         df = pandas.DataFrame(all_schemas)
#         df.columns = ["user_id", "time_of_detection"]
        
        if len(all_schemas):
            # 8 minute sequence, 6 minutes before, and 4 into the future
            t_prev = dt.timedelta(0, 0, 0, 0, 6)
            t_for = dt.timedelta(0, 0, 0, 0, 4)
            
            start_time = (all_schemas[0][1] - t_prev).strftime(SQL_TIMESTAMP)
            end_time = (all_schemas[0][1] + t_for).strftime(SQL_TIMESTAMP)
            
            sz_time = all_schemas[0][1].strftime(SQL_TIMESTAMP)
            
            sz_timestamps.append(sz_time)
            
#             print(all_schemas)
#             print(dt.timedelta(0, 0, 0, 0, 5))
#             print(all_schemas[0][1].strftime(SQL_TIMESTAMP))

In [55]:
sz_timestamps

['2017-11-08 14:32:33',
 '2017-11-28 18:31:33',
 '2017-12-15 05:11:53',
 '2018-02-23 07:08:35',
 '2018-02-23 10:29:34',
 '2018-06-22 02:21:35',
 '2018-06-29 11:46:48',
 '2018-07-04 13:27:31']

In [46]:
Query = "SELECT user_id, time_of_detection FROM seizure_detection ORDER BY time_of_detection ASC"
cur.execute(Query)

all_schemas = cur.fetchall()

df = pandas.DataFrame(all_schemas)
df.columns = ["user_id", "time_of_detection"]

631

In [76]:
true_fpr_list = 0
corrupted_dates = 0
for i in tqdm(range(len(df))):
    uid = df.iloc[i][0]
    fpr_time = df.iloc[i][1]
    
    if fpr_time.strftime(SQL_TIMESTAMP) not in sz_timestamps:
        true_fpr_list += 1
        # 8 minute sequence, 6 minutes before, and 4 into the future
        t_prev = dt.timedelta(0, 0, 0, 0, 6)
        t_for = dt.timedelta(0, 0, 0, 0, 4)

        start_time = (fpr_time - t_prev).strftime(SQL_TIMESTAMP)
        end_time = (fpr_time + t_for).strftime(SQL_TIMESTAMP)
        
        if "1970" in start_time:
            corrupted_dates += 1
            continue
        else:
            fp_descriptor = {
        'dataSource': 'pg-stage',
        'type': 'unknown',
        'userId': uid,
        'start': start_time,
        'end': end_time
    }
            link_query = ("select * from time_series_index where user_id = {0} " +
                      "and (started, ended) overlaps ('{1}', '{2}') " +
                      "order by started desc").format( uid, start_time, end_time)
            
            s3_links = stage_driver.queryRequest( link_query )
            data_collection = stage_s3_driver.MakeS3Requests( s3_links )
            
            data_acc, data_hr = (reformat_timeseries_pg( data_collection.epiSensorData.accelerometerData ),
                reformat_hr_pg( data_collection.epiSensorData.heartRateData ))
        
            t_acc, v_acc = data_acc[0], data_acc[1]
            t_hr, v_hr = data_hr[0], data_hr[1]

            export_data_seizure = {
                'descriptor': fp_descriptor,
                'data': {
                    'accelerometer': {
                        't': t_acc,
                        'v': v_acc
                    },
                    'heartRate': {
                        't': t_hr,
                        'v': v_hr
                    }
                }
            }
    
            save_intermediates( export_data_seizure, 'false-positive-data-raw',
                        run = cur_run,
                        user = fp_descriptor['userId'],
                        segment = i,
                        nested = True )
        
    else:
        continue
        

# Sanity check to make sure we haven't left in a seizure.
if true_fpr_list + len(sz_timestamps) == len(df):
    print("All seizure detections accounted for")
elif true_fpr_list + len(sz_timestamps) > len(df):
    print("There are seizure detections in data")
else:
    print("Too many values excluded from data")
    
print("There are {} corrupted dates".format(corrupted_dates))
print("Total false positives is: {}".format(true_fpr_list - corrupted_dates))

 20%|█▉        | 126/631 [00:00<00:00, 969.95it/s]

number of payloads: 0
Done.
number of payloads: 0
Done.
number of payloads: 0
Done.
number of payloads: 0
Done.
number of payloads: 0
Done.
number of payloads: 7
Done.
number of payloads: 11
Done.
number of payloads: 18
Done.
number of payloads: 10


 21%|██        | 134/631 [00:22<01:23,  5.95it/s] 

Done.
number of payloads: 13


 21%|██▏       | 135/631 [00:28<01:43,  4.78it/s]

...

Done.
number of payloads: 9


100%|█████████▉| 630/631 [24:36<00:02,  2.34s/it]

Done.
number of payloads: 8


100%|██████████| 631/631 [24:39<00:00,  2.34s/it]

Done.
All seizure detections accounted for
There are 125 corrupted dates
Total false positives is: 498


# Old Code

In [11]:
i = 17

data = seiz_log.iloc[i][4]
false_neg = seiz_log.iloc[i][7]
bad_sz = seiz_log.iloc[i][8]
atyp = seiz_log.iloc[i][9]
backend = seiz_log.iloc[i][10]

split_dat = data.split(' ')
# uid = split_dat[2]
# start = split_dat[7][2:] + ' ' + split_dat[8][:-2]
# end = split_dat[9][1:] + ' ' + split_dat[10][:-2]
# end

split_dat

# split_dat = data.split('&')
# start = split_dat[4][3:]
# end = split_dat[5][3:]

# uid = split_dat[0].split('?')[1][3:]

NameError: name 'seiz_log' is not defined

In [25]:
len(seiz_log)

22

In [68]:
with open('seizures_raw.json', 'r+') as fp:
    test = json.load(fp)
    test = test[:6]

X = seiz_log['Data']

for i in range(len(X[6:])):
#     test_str = str(X[i+6])
    string = str(X[i+6]).replace(u'\xa0', u' ')
    userID = string[10:15]
    
#     print(test_str)
    
    dat_string = string.split('OVERLAPS')[1]
    start_date = dat_string[3:22]
    end_date = dat_string[26:-2]
    
    data = {
        "damped": False,
        "dataSource": "pg-stage",
        "end": end_date,
        "faked": False,
        "occluded": False,
        "start": start_date,
        "userId": userID 
    }
    
    test.append(data)

with open('seizures_raw.json', 'wt') as fp:
    json.dump(test, fp, indent=4, separators=(',', ': '))

# False Positives

In [85]:
false_pos = pd.read_excel('False Positive Log_258.xlsx')
false_pos

Index(['Participant ', 'TOE', 'HR Baseline', 'HR Peak', 'Payloads', 'Offset',
       'Build'],
      dtype='object')

In [109]:
part = false_pos['Participant ']
data = false_pos['Payloads']
part = list(part)
data = list(data)

false_pos_dict = []
for i in range(len(part)):
    userID = data[0].split("user_id = ")[1][:5]
    start_date = data[0].split("OVERLAPS (\'")[1][:19]
    end_date = data[0].split("OVERLAPS (\'")[1].split(", \'")[1][:19]
    false_pos_dict.append(
    {
        "dataSource": "pg-stage",
        "end": end_date,
        "start": start_date,
        "userId": userID,
        "participant": part[i]
    })

with open("false_positives.json", 'w+') as nf:
    json.dump(false_pos_dict, nf, indent=4, separators=(',', ': '))